# Rag Process

- Step-1 : load the data

- Step-2 : split the data

- Step-3 : convert into vector embeddings

- Step-4 : store embeddings in vectordb

- Step-5 : when a new data comes again convert into vector

- Step-6 : apply cosine similarity with qnvector with all vectors stored in vectordb

- Step-7 : finally we will get the answer

# Rag Model

- Using LangChain and GeminiAI Embeddings model

**Installing Required Packages**

In [1]:
## All together
!pip install chromadb # is used to save vectors Database
!pip install langchain
!pip install langchain-community # chunking spliter
!pip install langchain-google-genai  # we can connect with gemini ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.5 MB/s eta 

**Step-1 : Loading the Data**

- Here we are loading text file

In [2]:
from langchain_community.document_loaders import TextLoader
dir(TextLoader)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'alazy_load',
 'aload',
 'lazy_load',
 'load',
 'load_and_split']

In [3]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader(file_path="/content/State_union (1).txt")
loader

In [4]:
raw_documents=loader.load()
raw_documents[0].page_content

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.\n\nLast year COVID-19 kept us apart. This year we are finally together again.\n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.\n\nWith a duty to one another to the American people to the Constitution.\n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.\n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated.\n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined.\n\nHe met the Ukrainian people.\n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.\n\nGroups of citizens blocking tanks with their bodi

**Step-2 : Splitting the data**

- We have two types of **Splitters**

  - 1.CharacterTextSplitter

  - 2.RecursiveCharacterTextSplitter

**1.CharacterTextSplitter**

**Basic Functionality:**

This splittere divides the text into chunks based solely on a character limit, regardless of natural text boundaries like sentences or paragraphs.

**Use Case:** it's suitable for simple scenarios where splitting the text by a fixed number of characters is sufficient, and you don't care about splitting within words or sentences.

**Key Parameters:**

- Chunk_size : The maximun number of characters in each chunk.

- chunk_overlap: The number of characters that should overlap between consecutive chunks.

**How it works:** it will take the input text and split it into parts by counting characters.It does not consider word boundaries,sentences,or paragraphs.This can lead to splits in awkward places,like the middle of a word or sentence

**2.Recursive text Splitter**

**Advance Functionaly:** This splitter splits the text recursively, starting with large,more meaninfuly,units like paragraphs,then working down to sentences,and then words if necessary.It aims to split at natural language boundaries when possible.

**Use Case:** this ideal when you want more readable chunks that preserve the integrity of sentences and paragraph but still need to adhere to a character or token limit.It's more flexible and avoids cutting off in the middel of words or sentences

**Key Parameters:**

- chunk_size: The maximum number of characters in each chunk.

- chunk_overlap: The number of characters that should overlap between consecutive chunks.

- separator: A list of characters use to separate the text (e.g.,Paragraphs,sentences).

**How it works:** It tries to split the text using the largest units first, like paragraphs or sentences.if the text is still too large after aplitting by these large units, it recursively splits the text down to smaller units like individual words,ensuring cleaner splits

In [ ]:
# Understanding about Without overlaping and with over lapping : with example


#   hello  how   are  you  im  going  to  movie
#     1     2     3    4    5     6    7    8
#     5     3     3    3    2      5   2    5    =  28

# i wan to divide 10 10 10
#   1st 10  : hello how ar
#   2nd 10  : e you im going t
#   remaining : o movie
# this is without overlaping

# ===============================================

# i wan to divide 10 10 10
#   1st 10  : hello how are you
#   2nd 10  : e you im going to mo
#   remaining : o movie
# this is with overlaping

# ===========================================================

# eg : do not hang till death
# do not
# hang till death
# this is without overlaping


# eg : do not hang till death
# do not hang
# not hang till death
# this is with overlaping

- Here we are using **CharacterTextSplitter**

In [6]:
# Split the documents into chunks
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0) # splitting the text into chunks
documents = text_splitter.split_documents(raw_documents)
print(documents[0].page_content)      # This is 1st document

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.

Last year COVID-19 kept us apart. This year we are finally together again.

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.

With a duty to one another to the American people to the Constitution.

And with an unwavering resolve that freedom will always triumph over tyranny.

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated.

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined.

He met the Ukrainian people.

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.


In [7]:
len([len(i.page_content) for i in documents])
# Text file splitted into number of documents     ===== 41 documents

41

In [8]:
print([len(i.page_content) for i in documents])
# in each document has number of chunks means characters less than 1000 ==== as we mentioned chunk_size=1000

[930, 980, 946, 935, 968, 943, 946, 974, 883, 992, 961, 924, 879, 982, 909, 961, 909, 914, 994, 941, 956, 882, 995, 942, 965, 963, 970, 874, 984, 936, 926, 908, 978, 907, 888, 895, 982, 998, 988, 999, 192]


**Step-3 : convert into vector embeddings**

- We have convert the text into embeddings

- With the help of any AI model we can convert text to embeddings

- Here we are using **GeminiAI** model to convert text to embeddings

In [9]:
# Set up embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings # GenAI model to convert words to embeddings
from google.colab import userdata
api_key=userdata.get('API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(
    model='models/embedding-001',
    google_api_key=api_key,       # here get your gemini api key
    task_type="retrieval_query"
)

**Step-4 : store embeddings in vectordb**

- We have to store the embeddings in a "vector DataBase"

- We have different types of "Vector DataBases"

  - Pinecone === login
  - Faiss   ==== open source
  - chromadb  ==== open scorces
  - Elasticsearch
  - waiat
  - redis

- Now in this model we are using **chromaDB** to store the embeddings in a vectorDB

In [10]:
# Create the vector store
from langchain.vectorstores import Chroma   # Store the vectors

# Create the vector store
vectordb = Chroma.from_documents(documents=documents, embedding=embeddings,
                                persist_directory='chromadb')
vectordb

**Step-5 : Getting Answer**

- Getting a new question

- Apply cosine similarity with qnvector with all vectors stored in vectordb

- Finally we will get the answer

In [11]:
query = "What did the president say about Ketanji Brown Jackson"
docs = vectordb.similarity_search(query)
print(docs[0].page_content)

In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections.

We cannot let this happen.

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections.

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service.

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


# All Together

- Rag_LangChain_GeminiAI_Embedding

In [ ]:
!pip install chromadb
!pip install langchain
!pip install langchain-community
!pip install langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.5 MB/s eta 

In [12]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader(file_path="/content/State_union (1).txt")
raw_documents=loader.load()


# Split the documents into chunks
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0) # splitting the text into chunks
documents = text_splitter.split_documents(raw_documents)


# Set up embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings # GenAI model to convert words to embeddings
from google.colab import userdata
api_key=userdata.get('API_KEY')


# GeminiAI Embedding Model
embeddings = GoogleGenerativeAIEmbeddings(
    model='models/embedding-001',
    google_api_key=api_key,       # here get your gemini api key
    task_type="retrieval_query"
)


# Create the vector store
from langchain.vectorstores import Chroma   # Store the vectors

# Create the vector store
vectordb = Chroma.from_documents(documents=documents, embedding=embeddings,
                                persist_directory='chromadb')

# New question
query = "What did the president say about Ketanji Brown Jackson"
docs = vectordb.similarity_search(query)
print(docs[0].page_content)

In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections.

We cannot let this happen.

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections.

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service.

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


# Rag Model

- Using LangChain and GeminiAI Embeddings model and GeminiAI Model

- Controlling the Rag model using prompting

**steps of RAG**

- load the data

- divide into chunks

- load the embedding model from GeminiAI

- Load the chat model of GeminiAI

- convert text into embedding vector : gemini ai , open ai , antropic , meta

- store into vectordb : chroma , pinecone , faiass

- prompt template

- retrival query chains

In [ ]:
## All together
!pip install chromadb # is used to save vectors Database
!pip install langchain
!pip install langchain-community # chunking spliter
!pip install langchain-google-genai  # we can connect with gemini ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.6 MB/s eta 

**Importing required packages**

In [13]:
from langchain.prompts import PromptTemplate # Prompt template
from langchain.vectorstores import Chroma   # Store the vectors
from langchain.text_splitter import RecursiveCharacterTextSplitter # Chunks
from langchain.document_loaders import TextLoader  # Load the text
from langchain.chains import RetrievalQA, LLMChain # Chains and Retrival ans
from langchain.retrievers.multi_query import MultiQueryRetriever # Multiple Answers
from langchain_google_genai import ChatGoogleGenerativeAI # GenAI model to retrive
from langchain_google_genai import GoogleGenerativeAIEmbeddings # GenAI model to convert words to embeddings

**Step-1 : Loading the data**

In [16]:
# Load documents
from langchain.document_loaders import TextLoader  # Load the text

loader = TextLoader('/content/State_union (1).txt')
raw_documents = loader.load()
raw_documents[0].page_content

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.\n\nLast year COVID-19 kept us apart. This year we are finally together again.\n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.\n\nWith a duty to one another to the American people to the Constitution.\n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.\n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated.\n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined.\n\nHe met the Ukrainian people.\n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world.\n\nGroups of citizens blocking tanks with their bodi

**Step-2 : Splitting the data**

- We have two types of **Splitters**

  - 1.CharacterTextSplitter

  - 2.RecursiveCharacterTextSplitter

**1.CharacterTextSplitter**

**Basic Functionality:**

This splittere divides the text into chunks based solely on a character limit, regardless of natural text boundaries like sentences or paragraphs.

**Use Case:** it's suitable for simple scenarios where splitting the text by a fixed number of characters is sufficient, and you don't care about splitting within words or sentences.

**Key Parameters:**

- Chunk_size : The maximun number of characters in each chunk.

- chunk_overlap: The number of characters that should overlap between consecutive chunks.

**How it works:** it will take the input text and split it into parts by counting characters.It does not consider word boundaries,sentences,or paragraphs.This can lead to splits in awkward places,like the middle of a word or sentence

**2.Recursive text Splitter**

**Advance Functionaly:** This splitter splits the text recursively, starting with large,more meaninfuly,units like paragraphs,then working down to sentences,and then words if necessary.It aims to split at natural language boundaries when possible.

**Use Case:** this ideal when you want more readable chunks that preserve the integrity of sentences and paragraph but still need to adhere to a character or token limit.It's more flexible and avoids cutting off in the middel of words or sentences

**Key Parameters:**

- chunk_size: The maximum number of characters in each chunk.

- chunk_overlap: The number of characters that should overlap between consecutive chunks.

- separator: A list of characters use to separate the text (e.g.,Paragraphs,sentences).

**How it works:** It tries to split the text using the largest units first, like paragraphs or sentences.if the text is still too large after aplitting by these large units, it recursively splits the text down to smaller units like individual words,ensuring cleaner splits

- Here we are using **RecursiveCharacterTextSpliter**

In [17]:
# Split the documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
print(documents)

[Document(metadata={'source': '/content/State_union (1).txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.\n\nLast year COVID-19 kept us apart. This year we are finally together again.\n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.\n\nWith a duty to one another to the American people to the Constitution.\n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.\n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated.\n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined.\n\nHe met the Ukrainian people.\n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determinati

In [18]:
len([len(i.page_content) for i in documents])
# Text file splitted into number of documents     ===== 41 documents

41

In [19]:
print([len(i.page_content) for i in documents])
# in each document has number of chunks means characters less than 1000 ==== as we mentioned chunk_size=1000

[930, 980, 946, 935, 968, 943, 946, 974, 883, 992, 961, 924, 879, 982, 909, 961, 909, 914, 994, 941, 956, 882, 995, 942, 965, 963, 970, 874, 984, 936, 926, 908, 978, 907, 888, 895, 982, 998, 988, 939, 252]


**Step-4: Set up the models**

- One is **embedding model**

  - We have to convert the text into embeddings

  - With the help of any AI model we can convert text to embeddings

  - Here we are using **GeminiAI** model to convert text to embeddings

- One is Chat model

  - We are using **GeminiAI** Model to control the rag model by providing the prompt


In [20]:
# Set up embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings # GenAI model to convert words to embeddings
from google.colab import userdata
api_key=userdata.get('API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(
    model='models/embedding-001',
    google_api_key=api_key,       # here get your gemini api key
    task_type="retrieval_query"
)

- GeminiAI chat model

In [21]:
# Set up chat model
from langchain_google_genai import ChatGoogleGenerativeAI # GenAI model to retrive
from google.generativeai.types.safety_types import HarmBlockThreshold, HarmCategory

# safety settings for chat model
safety_settings = {
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
                    }

# setting chat model
chat_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=api_key,
    temperature=0.3,   # Adjust the temparature here === means creativity value
    safety_settings=safety_settings
)


**Step-5: Get the Embeddings store in VectorDB**

- Here we are using **ChromaDB** to store the embeddings in vectorDB

In [22]:
# Create the vector store
from langchain.vectorstores import Chroma   # Store the vectors

vectordb = Chroma.from_documents(documents=documents, embedding=embeddings)
vectordb

**Step-6: Make the Prompt Template**

- To control the performance

In [23]:
from langchain.prompts import PromptTemplate # Prompt template

prompt_template = """
## Safety and Respect Come First!

You are programmed to be a helpful and harmless AI. You will not answer requests that promote:

* **Harassment or Bullying:** Targeting individuals or groups with hateful or hurtful language.
* **Hate Speech:**  Content that attacks or demeans others based on race, ethnicity, religion, gender, sexual orientation, disability, or other protected characteristics.
* **Violence or Harm:**  Promoting or glorifying violence, illegal activities, or dangerous behavior.
* **Misinformation and Falsehoods:**  Spreading demonstrably false or misleading information.

**How to Use You:**

1. **Provide Context:** Give me background information on a topic.
2. **Ask Your Question:** Clearly state your question related to the provided context.

**Please Note:** If the user request violates these guidelines, you will respond with:
"I'm here to assist with safe and respectful interactions. Your query goes against my guidelines. Let's try something different that promotes a positive and inclusive environment."

##  Answering User Question:

Answer the question as precisely as possible using the provided context. The context can be from different topics. Please make sure the context is highly related to the question. If the answer is not in the context, you only say "answer is not in the context".

Context: \n {context}
Question: \n {question}
Answer:
"""


prompt = PromptTemplate(template = prompt_template, input_variables=['context','question'])

In [24]:
print(prompt.template)


## Safety and Respect Come First!

You are programmed to be a helpful and harmless AI. You will not answer requests that promote:

* **Harassment or Bullying:** Targeting individuals or groups with hateful or hurtful language.
* **Hate Speech:**  Content that attacks or demeans others based on race, ethnicity, religion, gender, sexual orientation, disability, or other protected characteristics.
* **Violence or Harm:**  Promoting or glorifying violence, illegal activities, or dangerous behavior.
* **Misinformation and Falsehoods:**  Spreading demonstrably false or misleading information.

**How to Use You:**

1. **Provide Context:** Give me background information on a topic.
2. **Ask Your Question:** Clearly state your question related to the provided context.

**Please Note:** If the user request violates these guidelines, you will respond with:
"I'm here to assist with safe and respectful interactions. Your query goes against my guidelines. Let's try something different that promotes

**Step-7: Create tha QA chains**

In [25]:
# Create the QA
from langchain.retrievers.multi_query import MultiQueryRetriever # Multiple Answers
from langchain.chains import RetrievalQA, LLMChain # Chains and Retrival ans

retriever_from_llm = MultiQueryRetriever.from_llm(retriever=vectordb.as_retriever(search_kwargs={"k": 5}),
                                                  llm=chat_model)  # get top 5 answers from vectors

qa_chain = RetrievalQA.from_chain_type(llm=chat_model,
                                       retriever= retriever_from_llm,
                                       return_source_documents=True,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": prompt}
                                      )
# langchain : llm model , vectordb , prompt
# Retrivalqa means : It require all like llm chat model , vectorDB , Prompt

**Step-8 : Getting the question**

In [26]:
# Run the query
response = qa_chain.invoke({"What did the president say about Ketanji Brown Jackson?"})  # we are passing in dictionary because in prompt we have context in dictionary type
print(response)

{'query': {'What did the president say about Ketanji Brown Jackson?'}, 'result': "The president nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to the Supreme Court, calling her one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.  He also noted that since her nomination, she has received broad support from various groups, including the Fraternal Order of Police and former judges appointed by both Democrats and Republicans.", 'source_documents': [Document(metadata={'source': '/content/State_union (1).txt'}, page_content='In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections.\n\nWe cannot let this happen.\n\nTonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections.\n\nTonight, I’d like to honor someone who has dedicated his life to ser

In [27]:
response.keys()

dict_keys(['query', 'result', 'source_documents'])

In [28]:
response['query']

{'What did the president say about Ketanji Brown Jackson?'}

In [29]:
response['result']

"The president nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to the Supreme Court, calling her one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.  He also noted that since her nomination, she has received broad support from various groups, including the Fraternal Order of Police and former judges appointed by both Democrats and Republicans."

# All Together

- Rag_LangChain_GeminiAI_Embedding_GeminiAI_ChatModel

In [ ]:
!pip install chromadb
!pip install langchain
!pip install langchain-community
!pip install langchain-google-genai

In [30]:
from langchain.prompts import PromptTemplate # Prompt template
from langchain.vectorstores import Chroma   # Store the vectors
from langchain.text_splitter import RecursiveCharacterTextSplitter # Chunks
from langchain.document_loaders import TextLoader  # Load the text
from langchain.chains import VectorDBQA,RetrievalQA, LLMChain # Chains and Retrival ans
from langchain.retrievers.multi_query import MultiQueryRetriever # Multiple Answers
from langchain_google_genai import ChatGoogleGenerativeAI # GenAI model to retrive
from langchain_google_genai import GoogleGenerativeAIEmbeddings # GenAI model to conver words


# Load documents
# Load Excel data
# Load documents
from langchain.document_loaders import TextLoader  # Load the text

loader = TextLoader('/content/State_union (1).txt')
raw_documents = loader.load()

# Split the documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

# Set up embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings # GenAI model to convert words to embeddings
from google.colab import userdata
api_key=userdata.get('API_KEY')

# Set up embeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model='models/embedding-001',
    google_api_key=api_key,
    task_type="retrieval_query"
)



from google.generativeai.types.safety_types import HarmBlockThreshold, HarmCategory

safety_settings = {
HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,

}
chat_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=api_key,
    temperature=0.3,  # Adjust the temperature here
    safety_settings=safety_settings
)


# Create the vector store
vectordb = Chroma.from_documents(documents=documents, embedding=embeddings)

prompt_template = """
## Safety and Respect Come First!

You are programmed to be a helpful and harmless AI. You will not answer requests that promote:

* **Harassment or Bullying:** Targeting individuals or groups with hateful or hurtful language.
* **Hate Speech:**  Content that attacks or demeans others based on race, ethnicity, religion, gender, sexual orientation, disability, or other protected characteristics.
* **Violence or Harm:**  Promoting or glorifying violence, illegal activities, or dangerous behavior.
* **Misinformation and Falsehoods:**  Spreading demonstrably false or misleading information.

**How to Use You:**

1. **Provide Context:** Give me background information on a topic.
2. **Ask Your Question:** Clearly state your question related to the provided context.

**Please Note:** If the user request violates these guidelines, you will respond with:
"I'm here to assist with safe and respectful interactions. Your query goes against my guidelines. Let's try something different that promotes a positive and inclusive environment."

##  Answering User Question:

Answer the question as precisely as possible using the provided context. The context can be from different topics. Please make sure the context is highly related to the question. If the answer is not in the context, you only say "answer is not in the context".

Context: \n {context}
Question: \n {question}
Answer:
"""


prompt = PromptTemplate(template = prompt_template, input_variables=['context','question'])
# Create the QA
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=vectordb.as_retriever(search_kwargs={"k": 5}),
                                                  llm=chat_model)

qa_chain = RetrievalQA.from_chain_type(llm=chat_model,
                                       retriever= retriever_from_llm,
                                       return_source_documents=True,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": prompt}
                                      )

# Run the query
response = qa_chain.invoke({"What did the president say about Ketanji Brown Jackson?"})  # we are passing in dictionary because in prompt we have context in dictionary type
print(response['query'])
print(response['result'])

{'What did the president say about Ketanji Brown Jackson?'}
The president said he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to the Supreme Court four days prior.  He called her one of the nation's top legal minds and said she would continue Justice Breyer's legacy of excellence.  He also noted she received broad support from various groups, including the Fraternal Order of Police and former judges appointed by both Democrats and Republicans.


In [33]:
response['query']

{'What did the president say about Ketanji Brown Jackson?'}

In [32]:
response['result']

"The president said he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to the Supreme Court four days prior.  He called her one of the nation's top legal minds and said she would continue Justice Breyer's legacy of excellence.  He also noted she received broad support from various groups, including the Fraternal Order of Police and former judges appointed by both Democrats and Republicans."